In [2]:
import pandas as pd

# Iniciamos abriendo el archivo JSON con el objetivo de transformarlo en un dataframe.
with open('australian_user_reviews.json', 'r', encoding='utf-8') as file:
    data = [eval(line) for line in file]

user_reviews = pd.DataFrame(data)

# Ampliamos la columna 'reviews' con el propósito de convertir cada elemento de la lista en una fila
user_reviews_exploded = user_reviews.explode('reviews')

# Utilizamos la función json_normalize para desplegar los diccionarios contenidos en la columna 'reviews
user_reviews_normalized = pd.json_normalize(user_reviews_exploded['reviews'])

# Restablecimos el índice en ambas DataFrames.
user_reviews_exploded.reset_index(drop=True, inplace=True)
user_reviews_normalized.reset_index(drop=True, inplace=True)

# Se busca conservar las columnas originales de 'user_reviews' además de las 
# nuevas columnas generadas a partir de 'reviews', logrando esto mediante una fusión (merge) basada en el índice.
user_reviews_final = pd.concat([user_reviews_exploded.drop('reviews', axis=1), user_reviews_normalized], axis=1)

# El tamaño del DataFrame 'user_reviews_final' es el siguiente:
num_filas, num_columnas = user_reviews_final.shape

# Muestra el tamaño del DataFrame.
print(f"Número de filas: {num_filas}")
print(f"Número de columnas: {num_columnas}")

Número de filas: 59333
Número de columnas: 9


In [4]:
import pandas as pd

# Para analizar los datos del archivo JSON, primero debemos abrirlo y convertirlo en un dataframe.
with open('australian_users_items.json', 'r', encoding='utf-8') as file:
    data = [eval(line) for line in file]

user_items = pd.DataFrame(data)

# Para analizar los datos de la columna "items", primero debemos expandirla para que cada elemento de la lista sea una fila.
user_items_exploded = user_items.explode('items')

# Para convertir los diccionarios en la columna "items" a series de filas, podemos usar la función json_normalize(). 
# Esta función convertirá los diccionarios en series de filas, una para cada elemento del diccionario.
user_items_normalized = pd.json_normalize(user_items_exploded['items'])

# Reiniciamos el índice en ambos DataFrames
user_items_exploded.reset_index(drop=True, inplace=True)
user_items_normalized.reset_index(drop=True, inplace=True)

# Para evitar conflictos entre los índices de ambos DataFrames, primero debemos reiniciarlos.
user_items_final = pd.concat([user_items_exploded.drop('items', axis=1), user_items_normalized], axis=1)

# El número de filas y columnas del dataframe user_items_final es:
num_filas, num_columnas = user_items_final.shape

# Imprime el tamaño del DataFrame
print(f"Número de filas: {num_filas}")
print(f"Número de columnas: {num_columnas}")

Número de filas: 5170015
Número de columnas: 8


In [5]:
import pandas as pd
import json

# Declarar una variable de tipo list para almacenar los objetos JSON individuales.
data = []

# Abrir el archivo JSON y cargar cada objeto JSON por separado
with open('output_steam_games.json', 'r', encoding='utf-8') as file:
    for line in file:
        json_data = json.loads(line)
        data.append(json_data)

# Convertir la lista de objetos JSON en un DataFrame
steam_games = pd.DataFrame(data)

# El tamaño del dataframe user_items_final:
num_filas, num_columnas = steam_games.shape

# Imprime el tamaño del DataFrame
print(f"Número de filas: {num_filas}")
print(f"Número de columnas: {num_columnas}")

Número de filas: 120445
Número de columnas: 19


In [6]:
# Generar un resumen de los datos nulos o faltantes en cada columna.
datos_nulos = steam_games.isnull().sum()

# Imprime la cantidad de datos nulos por columna
print(datos_nulos)

publisher          96362
genres             91593
app_name           88312
title              90360
url                88310
release_date       90377
tags               88473
reviews_url        88312
discount_price    120220
specs              88980
price              89687
early_access       88310
id                 88312
metascore         117768
developer          91609
user_id            32135
steam_id           32135
items              32135
items_count        32135
dtype: int64


In [7]:
# Se eliminan los datos faltantes de la columna "url", ya que esta columna es irrelevante y posee más del 90% de datos faltantes.
steam_games = steam_games.dropna(subset=['url'])

# Se renombro la columna 'id' a 'item_id'
steam_games.rename(columns={'id': 'item_id'}, inplace=True)

# Aplana la columna "genres" del DataFrame "steam_games".
steam_games = steam_games.explode('genres', ignore_index=True)

# Se verifica el tamaño del dataframe steam_games.
num_filas, num_columnas = steam_games.shape

# Imprime el tamaño del DataFrame
print(f"Número de filas: {num_filas}")
print(f"Número de columnas: {num_columnas}")

Número de filas: 74837
Número de columnas: 19


In [31]:
# Se agrupan los datos de las columnas 'item_id' con el fin de calcular la suma de la columna 'playtime_forever'.

grouped_user_items = user_items_final.groupby('item_id')['playtime_forever'].sum().reset_index()

# Se renombro la columna de playtime_forever.
grouped_user_items.rename(columns={'playtime_forever': 'sum_playtime_forever'}, inplace=True)

# El tamaño del nuevo dataframe grouped_user_items:
num_filas, num_columnas = grouped_user_items.shape

# Imprime el tamaño del DataFrame.
print(f"Número de filas: {num_filas}")
print(f"Número de columnas: {num_columnas}")

Número de filas: 10978
Número de columnas: 2


In [33]:
# Se concatenan los DataFrames grouped_user_items con user_reviews con el objetivo de tener toda la información en una sola tabla.
merged_df = pd.merge( grouped_user_items, user_reviews_final, on='item_id', how='inner')


# El tamaño del nuevo dataframe grouped_user_items:
num_filas, num_columnas = merged_df.shape

# Imprime el tamaño del DataFrame
print(f"Número de filas: {num_filas}")
print(f"Número de columnas: {num_columnas}")

Número de filas: 52781
Número de columnas: 10


In [34]:
# Se concatenan los dataframes merged_df con steam_games con la finalidad de poseer la informacion en una sola tabla
merged_final = pd.merge( merged_df, steam_games, on='item_id', how='inner')

# El tamaño del nuevo dataframe grouped_user_items:
num_filas, num_columnas = merged_final.shape

# Imprime el tamaño del DataFrame
print(f"Número de filas: {num_filas}")
print(f"Número de columnas: {num_columnas}")

Número de filas: 118376
Número de columnas: 28


In [11]:
# Se validan los nombres de las columnas para luego eliminar aquellas que se consideren irrelevantes para la creación de los endpoints y el sistema de recomendación.

nombres_columnas = merged_final.columns
print(nombres_columnas)

Index(['item_id', 'sum_playtime_forever', 'user_id_x', 'user_url', 'funny',
       'posted', 'last_edited', 'helpful', 'recommend', 'review', 'publisher',
       'genres', 'app_name', 'title', 'url', 'release_date', 'tags',
       'reviews_url', 'discount_price', 'specs', 'price', 'early_access',
       'metascore', 'developer', 'user_id_y', 'steam_id', 'items',
       'items_count'],
      dtype='object')


In [14]:
# Puedes eliminar las columnas especificadas de la siguiente manera:

columnas_a_eliminar = ['user_id_y', 'user_url', 'funny', 'last_edited', 'helpful', 'publisher', 'discount_price', 'specs', 'price', 'early_access', 'metascore', 'items_count','items','url','reviews_url']
merged_final= merged_final.drop(columns=columnas_a_eliminar)


# Se renombra la columna 'user_id_y' a 'user_id' de la siguiente manera:

merged_final = merged_final.rename(columns={'user_id_x': 'user_id'})

In [15]:
# Conversión de datos de las columnas 'posted'.

import re

# Se cuenta con un DataFrame llamado 'merged_final'

# Define una función para extraer la fecha usando regex
def extract_date(text):
    match = re.search(r'\w+ \d{1,2}, \d{4}', text)
    if match:
        return match.group()

# Aplica la función solo a las filas donde 'posted' no sea de tipo Timestamp
mask = merged_final['posted'].apply(lambda x: isinstance(x, str))
merged_final.loc[mask, 'posted'] = merged_final.loc[mask, 'posted'].apply(lambda x: extract_date(x))

# Convierte la columna 'posted' a formato de fecha
merged_final['posted'] = pd.to_datetime(merged_final['posted'], format='%B %d, %Y')

In [16]:
#Se procede a validar la cantidad de datos faltantes dentro del dataframe resultante para realizar imputación

# Se valida los datos nulos de las columnas
nulos_por_columna = merged_final.isnull().sum()
print(nulos_por_columna)

item_id                      0
sum_playtime_forever         0
user_id                      0
posted                   21786
recommend                    0
review                       0
genres                    1963
app_name                     0
title                     1783
release_date              1915
tags                         0
developer                 2130
steam_id                118376
dtype: int64


In [17]:
# Realizaremos la imputación de los valores faltantes en las columnas del DataFrame.

# Para las columnas 'genres' y 'title', que contienen información de tipo texto, utilizaremos "sin dato" como mensaje descriptivo.

merged_final['genres'].fillna("sin dato", inplace=True)
merged_final['title'].fillna("sin dato", inplace=True)
merged_final['developer'].fillna("sin dato", inplace=True)

# Imputación de datos de las columnas user_id y steam_id, para ello se asigno un valor unico -1

merged_final['user_id'].fillna(-1, inplace=True)
merged_final['steam_id'].fillna(-1, inplace=True)



# Las columnas 'release_date' y 'posted', contienen información de tipo date, se considero imputar calculando el valor máximo de la columna release_date de acuerdo al item_id

# Agrupa por 'item_id' y encuentra el valor máximo de 'posted' para cada grupo
max_posted_by_item = merged_final.groupby('item_id')['release_date'].max()

# Si deseas convertirlo en un diccionario para acceder al valor máximo de 'posted' por 'item_id', puedes hacerlo de esta manera:
max_posted_dict = max_posted_by_item.to_dict()

# Reemplaza los valores faltantes en 'posted' con el valor máximo correspondiente según 'item_id'
merged_final['release_date'].fillna(merged_final['item_id'].map(max_posted_dict), inplace=True)


# Finalmente me estan quedando 1599 valores sin reemplazar. Que fueron reemplazados por el valor modal de la columna 

# Limpiar la columna 'release_date' eliminando valores no válidos
merged_final['release_date'] = pd.to_datetime(merged_final['release_date'], errors='coerce', format="%Y-%m-%d")

# Calcular la moda de la columna 'release_date' sin eliminar los valores nulos
moda_release_date = merged_final['release_date'].mode()

# Obtener la primera moda (en caso de que haya múltiples modas)
moda_release_date_primera = moda_release_date.iloc[0]

# Reemplazar los valores faltantes en 'release_date' con la moda
merged_final['release_date'].fillna(moda_release_date_primera, inplace=True)


# Imputado de la columna 'posted', contienen información de tipo date, se considero imputar calculando el valor máximo de la columna release_date de acuerdo al item_id

# Agrupa por 'item_id' y encuentra el valor máximo de 'posted' para cada grupo
max_posted_by_item = merged_final.groupby('item_id')['posted'].max()

# Si deseas convertirlo en un diccionario para acceder al valor máximo de 'posted' por 'item_id', puedes hacerlo de esta manera:
max_posted_dict = max_posted_by_item.to_dict()

# Reemplaza los valores faltantes en 'posted' con el valor máximo correspondiente según 'item_id'
merged_final['posted'].fillna(merged_final['item_id'].map(max_posted_dict), inplace=True)


# Finalmente me estan quedando 1723 valores sin reemplazar. Que fueron reemplazados por el valor modal de la columna 

# Limpiar la columna 'release_date' eliminando valores no válidos
merged_final['posted'] = pd.to_datetime(merged_final['posted'], errors='coerce', format="%Y-%m-%d")

# Calcular la moda de la columna 'release_date' sin eliminar los valores nulos
moda_release_date = merged_final['posted'].mode()

# Obtener la primera moda (en caso de que haya múltiples modas)
moda_release_date_primera = moda_release_date.iloc[0]

# Reemplazar los valores faltantes en 'release_date' con la moda
merged_final['posted'].fillna(moda_release_date_primera, inplace=True)

In [18]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Se realiza el análisis de sentimiento de la columna reviews

# Descargar el léxico VADER si aún no está descargado
nltk.download('vader_lexicon')

# Crear una instancia del analizador de sentimiento VADER
analyzer = SentimentIntensityAnalyzer()

# Función para asignar un valor de sentimiento según la escala dada
def assign_sentiment(row):
    # Realizar el análisis de sentimiento en el texto de la columna 'review'
    sentiment_scores = analyzer.polarity_scores(row['review'])
    
    # Calcular un puntaje de sentimiento compuesto
    sentiment_score = sentiment_scores['compound']
    
    # Asignar valores según la escala
    if sentiment_score < -0.05:
        return 0  # Sentimiento malo
    elif sentiment_score >= -0.05 and sentiment_score <= 0.05:
        return 1  # Sentimiento neutral o sin dato
    else:
        return 2  # Sentimiento positivo

# Aplicar la función a cada fila del DataFrame y crear la columna 'sentiment_analysis'
merged_final['sentiment_analysis'] = merged_final.apply(assign_sentiment, axis=1)

# Eliminar la columna 'review'
merged_final = merged_final.drop('review', axis=1)

# Contar los registros por valor en la columna 'sentiment_analysis'
sentiment_counts = merged_final['sentiment_analysis'].value_counts()

# Mostrar el resultado
print(sentiment_counts)

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


sentiment_analysis
2    75004
1    23298
0    20074
Name: count, dtype: int64


In [19]:
# Se visualiza el tipo de dato de las columnas de la siguiente manera:

# Se corrobora el tipo de dato de las columnas del dataframe
tipos_de_dato = merged_final.dtypes

# Se Imprime los tipos de datos
print(tipos_de_dato)

item_id                         object
sum_playtime_forever           float64
user_id                         object
posted                  datetime64[ns]
recommend                       object
genres                          object
app_name                        object
title                           object
release_date            datetime64[ns]
tags                            object
developer                       object
steam_id                         int64
sentiment_analysis               int64
dtype: object


In [20]:
# Convierte la columna 'recommend' a tipo booleano
merged_final['recommend'] = merged_final['recommend'].astype(bool)

In [21]:
#Se procede a validar la cantidad de datos faltantes dentro del dataframe resultante para realizar imputación

# Se valida los datos nulos de las columnas
nulos_por_columna = merged_final.isnull().sum()
print(nulos_por_columna)

item_id                 0
sum_playtime_forever    0
user_id                 0
posted                  0
recommend               0
genres                  0
app_name                0
title                   0
release_date            0
tags                    0
developer               0
steam_id                0
sentiment_analysis      0
dtype: int64


In [22]:
# Se crea y valida que las columnas que involucran fecha se encuentren en el formato correcto

# Convierte la columna 'release_date' y 'posted' al tipo de dato datetime
merged_final['release_date'] = pd.to_datetime(merged_final['release_date'], errors='coerce', format="%B %d, %Y")
merged_final['posted'] = pd.to_datetime(merged_final['posted'], errors='coerce', format="%B %d, %Y")

# Verifica si hay errores en la conversión
print(merged_final['release_date'].isnull().sum())  # Debería imprimir 0 si no hay errores
print(merged_final['posted'].isnull().sum())       # Debería imprimir 0 si no hay errores

# Crea la columna 'anio_release' que contiene el año de 'release_date'
merged_final['anio_release'] = merged_final['release_date'].dt.year

# Crea la columna 'anio_posted' que contiene el año de 'posted'
merged_final['anio_posted'] = merged_final['posted'].dt.year

0
0


In [25]:
# Se crea dataset para ser empleada en el primer endpoints que se consumirán en la API
# se consideraron las columnas de genres (genero), sum_playtime_forever (horas jugadas), anio_release (año de lanzamiento)

dataset_f1=merged_final[["genres", "sum_playtime_forever", "anio_release"]]

# Se pasa el dataframe creado a csv:

dataset_f1.to_csv('dataset_f1.csv', index=False)

In [26]:
# Se crea dataset para ser empleada en el segundo endpoints que se consumirán en la API
# se consideraron las columnas de genres (genero), sum_playtime_forever (horas jugadas), anio_posted (año de posteo, se considera el año que jugo el usuario), user_id (identificador unico de usuario)

dataset_f2=merged_final[["genres", "sum_playtime_forever", "anio_posted","user_id"]]

# Se pasa el dataframe creado a csv:

dataset_f2.to_csv('dataset_f2.csv', index=False)

In [27]:
# Se crea dataset para ser empleada en el tercero y cuarto endpoints que se consumirán en la API
# se consideraron las columnas de genres

dataset_f3=merged_final[["user_id", "app_name", "anio_posted","recommend"]]

# Se pasa el dataframe creado a csv:

dataset_f3.to_csv('dataset_f34.csv', index=False)

In [28]:
# Se crea dataset para ser empleada en el quinto endpoints que se consumirán en la API
# se consideraron las columnas de genres

dataset_f5=merged_final[["sentiment_analysis", "anio_release"]]

# Se pasa el dataframe creado a csv:

dataset_f5.to_csv('dataset_f5.csv', index=False)

In [28]:
# Se crea dataset con las columnas necesarias para el modelo de machine learning.

dataset_model=merged_final[["title", "user_id","tags","genres","recommend"]]


# Se calcula el porcentaje de duplicados en la columna "title"
duplicados = dataset_model['title'].duplicated(keep=False)
porcentaje_duplicados = (duplicados.sum() / len(dataset_model)) * 100

# 'duplicados' es una serie booleana que indica qué filas son duplicadas
# 'porcentaje_duplicados' calcula el porcentaje de duplicados con respecto al total de filas en el DataFrame

print(f'Porcentaje de duplicados en la columna "title": {porcentaje_duplicados:.2f}%')

# Se elimina los duplicado de la columna title con la finalidad de reducir recursos computacionales y mejorar recursos computacionales para el modelo
dataset_model.drop_duplicates(subset="title", inplace=True)

dataset_model

# Se pasa el dataframe creado a csv:

dataset_model.to_csv('modelo.csv', index=False)


Porcentaje de duplicados en la columna "title": 99.80%


C:\Users\Usuario\AppData\Local\Temp\ipykernel_10120\3551264269.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_model.drop_duplicates(subset="title", inplace=True)
